In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 256
NUM_EPOCHS = 40

# Architecture
NUM_CLASSES = 10

# Other
DEVICE = "cuda:0"

In [3]:
train_indices = torch.arange(0, 48000)
valid_indices = torch.arange(48000, 50000)


train_transform = transforms.Compose([transforms.Resize((70, 70)),
                                      transforms.RandomCrop((64, 64)),
                                      transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((70, 70)),
                                     transforms.CenterCrop((64, 64)),
                                     transforms.ToTensor()])

train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=train_transform,
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)
test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=test_transform,
                                download=False)




train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE,
                          num_workers=4,
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=4,
                         shuffle=False)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [4]:
# Checking the dataset
print('Training Set:\n')
for images, labels in train_loader:
  print('Image batch dimensions:', images.size())
  print('Image label dimensions:', labels.size())
  print(labels)
  break

# Checking the dataset
print('\nValidation Set:\n')
for images, labels in valid_loader:
  print('Image batch dimensions:', images.size())
  print('Image label dimensions:', labels.size())
  print(labels)
  break

# Checking the dataset
print('\nTesting Set:\n')
for images, labels in test_loader:
  print('Image batch dimensions:', images.size())
  print('Image label dimensions:', labels.size())
  print(labels)
  break

Training Set:



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Image batch dimensions: torch.Size([256, 3, 64, 64])
Image label dimensions: torch.Size([256])
tensor([9, 6, 7, 5, 0, 1, 1, 2, 9, 8, 1, 0, 1, 8, 0, 3, 8, 3, 2, 0, 6, 3, 8, 7,
        7, 1, 1, 7, 2, 4, 4, 8, 5, 3, 3, 9, 5, 8, 8, 9, 8, 7, 5, 4, 1, 0, 8, 5,
        8, 4, 0, 9, 7, 8, 1, 8, 9, 1, 8, 0, 3, 6, 5, 6, 1, 7, 4, 5, 7, 1, 7, 8,
        1, 8, 2, 5, 6, 5, 3, 5, 9, 3, 8, 9, 3, 3, 0, 8, 5, 9, 7, 0, 0, 5, 0, 7,
        8, 0, 4, 2, 7, 4, 2, 6, 1, 9, 7, 7, 4, 4, 0, 6, 8, 7, 6, 1, 2, 0, 8, 8,
        1, 5, 6, 3, 3, 4, 6, 4, 8, 0, 2, 5, 6, 3, 7, 7, 1, 1, 5, 2, 9, 4, 6, 4,
        3, 8, 6, 5, 3, 2, 8, 4, 1, 3, 4, 7, 0, 5, 3, 7, 4, 0, 5, 0, 5, 5, 5, 2,
        1, 7, 4, 6, 9, 0, 0, 5, 5, 6, 6, 5, 4, 4, 7, 5, 4, 6, 4, 1, 3, 6, 9, 7,
        9, 5, 7, 4, 5, 7, 0, 3, 5, 1, 1, 6, 6, 8, 4, 6, 8, 2, 8, 8, 5, 2, 2, 1,
        7, 6, 0, 9, 4, 7, 2, 6, 4, 7, 7, 5, 6, 6, 2, 8, 0, 2, 0, 1, 2, 3, 3, 4,
        7, 9, 6, 9, 9, 2, 6, 8, 7, 7, 8, 2, 7, 8, 4, 7])

Validation Set:

Image batch dimensions: torch.

In [5]:
##########################
### MODEL
##########################

class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        #features : 특징을 추출하는 layers : 총 5번의 컨볼루션 연산을 거친다
        self.features = nn.Sequential(
            #1. 3 -> 64채널로 컨볼루션 연산 + MaxPolling
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            #2. 64 -> 192채널로 컨볼루션 연산 + MaxPolling
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            #3. 192 -> 384채널로 컨볼루션 연산
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #4. 384 -> 256채널로 컨볼루션 연산
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #5. 256 -> 256채널로 컨볼루션 연산 + MaxPolling
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        #avgpool : 풀링 기법 중 하나, avgpooling을 하면 텐서 모양을 classifier에 맞춰준다.
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        #classifier : 추출된 특징을 기반으로 데이터를 분류하는 layers
        self.classifier = nn.Sequential
        (
            nn.Dropout(0.5),
            #1. 256x6x6 의 특징을, 4096개로 압축한다.
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            #2. 4096개의 특징 차원은 유지하되 한번더 정제한다.
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            #4. 차원을 클래수 개수만큼으로 축소시키면서 클래스 분류를 완료한다. 
            nn.Linear(4096, num_classes)
        )
        #forward : 순전파함수
    def forward(self, x):
        #1. 먼저 features(x)를 통해, x의 특징을 추출한다.
        x = self.features(x)
        #2. 평균화작업을 하면서, 데이터의 모양을 맞춘다.
        x = self.avgpool(x)
        #3. classifer를 통해 클래스를 분류하고 softmax로 가장 확률이 높은 클래스를 선택하게 한다.
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas


In [ ]:
torch.manual_seed(RANDOM_SEED)

model = AlexNet(NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [ ]:
def compute_acc(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    #eval() : 모델을 평가모드로 전환
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        #features : 데이터가 담겨있는 텐서
        features = features.to(device)
        #targets : 정답이 담겨있는 텐서 
        targets = targets.to(device)
        #probas : 모델이 예측한 각 클래스에 속할 확률을 담고 있음
        logits, probas = model(features)
        #predicted_labels : 모델이 예측한 정답
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        #맞은 경우를 세서 확률로 만들어 줌
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    
#time.time() 현재 시간을 기록함
start_time = time.time()

cost_list = []
train_acc_list, valid_acc_list = [], []


for epoch in range(NUM_EPOCHS):
    #모델을 학습모드로 바꾼다.
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        #모델을 통해 예측을 수행하고
        logits, probas = model(features)
        #loss인 cross_entropy를 구한다.
        cost = F.cross_entropy(logits, targets)
        #backpropagation 전에는 optimizer의 남아있는 기울기를 제거
        optimizer.zero_grad()
        #backpropagation 수행
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        #################################################
        ### CODE ONLY FOR LOGGING BEYOND THIS POINT
        ################################################
        cost_list.append(cost.item())
        if not batch_idx % 150:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
                   f' Cost: {cost:.4f}')

        
    #모델을 평가모드로 바꾸기
    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        train_acc = compute_acc(model, train_loader, device=DEVICE)
        valid_acc = compute_acc(model, valid_loader, device=DEVICE)
        #validation 데이터셋에서 평가를 수행해본다.
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d}\n'
              f'Train ACC: {train_acc:.2f} | Validation ACC: {valid_acc:.2f}')
        
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)
    
    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')
  
elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#plt.plot()을 통해 시각화한다
plt.plot(cost_list, label='Minibatch cost')
plt.plot(np.convolve(cost_list, 
                     np.ones(200,)/200, mode='valid'), 
         label='Running average')

plt.ylabel('Cross Entropy')
plt.xlabel('Iteration')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(1, NUM_EPOCHS+1), train_acc_list, label='Training')
plt.plot(np.arange(1, NUM_EPOCHS+1), valid_acc_list, label='Validation')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
with torch.set_grad_enabled(False):
    test_acc = compute_acc(model=model,
                           data_loader=test_loader,
                           device=DEVICE)
    
    valid_acc = compute_acc(model=model,
                            data_loader=valid_loader,
                            device=DEVICE)
    

print(f'Validation ACC: {valid_acc:.2f}%')
print(f'Test ACC: {test_acc:.2f}%')